In [ ]:
#Se instala la herramienta RDKit
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 31.6 MB/s eta 0:00:00


In [ ]:
#Se instala el paquete duckdb que permite manipular la BD que inicialmente es muy grande
!pip install duckdb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Se hace instalación de paquete de kaggle
!pip install -q kaggle

In [ ]:
# Se usa la extensión de google colab para subir el token descargado de Kaggle el cual tiene el nombre "kaggle.json"
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"lhcastrop","key":"56e17656944c982f8a2a8576a52759d1"}'}

In [ ]:
#Se crea directorio
!mkdir ~/.kaggle

In [ ]:
#Se copia archivo en carpeta de colab
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Se carga el dataset desde kaggle en formato comprimido
! kaggle competitions download -c 'leash-BELKA'

100% 4.16G/4.16G [02:45<00:00, 26.1MB/s]
100% 4.16G/4.16G [02:45<00:00, 27.0MB/s]


In [ ]:
#Se crea un directorio
!mkdir leash-BELKA

In [ ]:
#Se descomprimen los archivos en el directorio creado
! unzip leash-BELKA.zip -d train

Archive:  leash-BELKA.zip
  inflating: train/sample_submission.csv  
  inflating: train/test.csv          
  inflating: train/test.parquet      
  inflating: train/train.csv         
  inflating: train/train.parquet     


**Preparación de Datos**

Los paquetes de datos de entrenamiento y prueba son definidos en los archivos .parquet. Se usa la herramienta duckdb para escanear la busqueda a traves de conjuntos de entramiento grandes. Para comenzar solo se toman un numero igual de muestras positivas y negativas.

Esta busqueda selecciona un numero igual de muestras donde los enlaces son 0 (no-enlazado) y 1 (enlazado), limitado para 30000 cada uno para evitar el sesgo del modelo hacia una clase particular.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import duckdb
import pandas as pd

train_path = '/content/train/train.parquet'
test_path = '/content/train/test.parquet'


con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

Mounted at /content/drive


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,186434414,O=C(Nc1ccc(C(=O)O)c([N+](=O)[O-])c1)OCC1c2cccc...,CCSCCN.Cl,Cl.Cl.NCc1cccc(-n2ccnn2)c1,CCSCCNc1nc(NCc2cccc(-n3ccnn3)c2)nc(Nc2ccc(C(=O...,sEH,0
1,101534511,O=C(N[C@@H](Cc1ccc([N+](=O)[O-])cc1)C(=O)O)OCC...,COC1(CN)CCOC1.Cl,Nc1cccnc1Cl,COC1(CNc2nc(Nc3cccnc3Cl)nc(N[C@@H](Cc3ccc([N+]...,BRD4,0
2,84271255,O=C(NCc1cccc(C(=O)O)c1)OCC1c2ccccc2-c2ccccc21,Cc1ccc(-c2cc(C(F)(F)F)nc(OCCN)n2)cc1,NCc1cnns1,Cc1ccc(-c2cc(C(F)(F)F)nc(OCCNc3nc(NCc4cccc(C(=...,HSA,0
3,284680516,O=C(O)[C@@H]1Cc2ccc(O)cc2CN1C(=O)OCC1c2ccccc2-...,C[Si](C)(C)C#Cc1ccc(N)cn1,Nc1nc2ccc(Cl)cc2s1,C[Si](C)(C)C#Cc1ccc(Nc2nc(Nc3nc4ccc(Cl)cc4s3)n...,HSA,0
4,31106779,COc1cc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3ccccc32)cn1,Cl.NCCC[C@@H]1NC(=O)NC1=O,Cc1nnc(N)o1,COc1cc(C(=O)N[Dy])c(Nc2nc(NCCC[C@@H]3NC(=O)NC3...,HSA,0


**Procesamiento de Caracteristicas**

Se toman los SMILEs correspondientes a cada molecula completamente ensamblada (molecule_smiles) y se generan los ECFPs para estas. Podriamos escoger diferentes radios o bits, pero 2 y 1024 son los mas estandar.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

# Se convierten los SMILEs en moleculas RDKit
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

#Se generan las ECFPs (Extended-Conectivity Fingerprints- Huellas Dactilares de Conectividad Extendida)
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use MorganGenerator
[03:26:10] DEPRECATION WARNING: please use Morga

**Modelo de Entrenamiento**


In [ ]:
# Se realiza la codificación One-hot a los nombres de las proteinas almacenadas en protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Se combinan las ECFPs y los nombres de la proteinas codificadas
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Se dividen los datos en conjunto de prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Se crea y entrena el Modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Se realizan las predicciones  con el conjunto de prueba
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probabilidad de la clase positiva

# Se calcula la precisión media promedio
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.96


**Prueba de Predicción**

El modelo Random Forest entrenado es usado para predecir las probabilidad de afinidad. Estas predicciones se guardan en un archivo CSV.

In [ ]:
import os
# Se procesa el archivo test.parquet fragmento por fragmento
test_file = '/content/train/test.csv'
output_file = 'submission.csv'  #  Se especifica la ruta y el nombre del archivo de salida
# Se lee el archivo test.parquet en un Dataframe de pandas
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Se generan las ECFPs para las molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # Codificacion One-hot para los nombres de las proteina almacenas en protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Se combinan las ECFPs y los nombres de la proteinas codificadas
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Se predicen las probabilidades
    probabilities = rf_model.predict_proba(X_test)[:, 1]

    # Se crea el DataFrame con columnas de 'id' y 'probabilidad'
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Almacenar el DataFrame de salida en un archivo CSV
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))

Se truncaron las últimas líneas 5000 del resultado de transmisión.
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use MorganGenerator
[04:34:55] DEPRECATION WARNING: please use Morga